# Validaciones - Pobreza

#### Tablas 2010 - Personas - Total Pais

- Poblacion y porcentaje por Pobreza SI
- Poblacion y porcentaje por Indigencia SI


- Poblacion y porcentaje por Pobreza SI y AGLO SI
- Poblacion y porcentaje por Indigencia SI y AGLO SI


- Poblacion y porcentaje Por Pobreza SI y Aglomerado
- Poblacion y porcentaje Por Indigencia SI y Aglomerado


- Poblacion y porcentaje Por Pobreza SI y Provincia
- Poblacion y porcentaje Por Indigencia SI y Provincia



In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
from IPython.display import display, Markdown, Latex

import glob
frac = 0.01

path ='./../data/Pobreza/pobreza_' # use your path

allFiles = []
allFiles += glob.glob(path+str(frac)+'_q*.csv')
allFiles = sorted(allFiles)
# allFiles

In [4]:
aglo_labels = pd.read_csv('./../data/info/aglo_labels.csv')

In [9]:
df_list = []
for f in allFiles:
    print(f)
    
    groupers = ['AGLOMERADO', 'PROV', 'NOMPROV']
    vars_ = ['Pobreza', 'Indigencia']
    
    data = pd.read_csv(f, usecols = ['Q'] + groupers + vars_)
    data = data.merge(aglo_labels)
    data['AGLO_si'] = data.AGLOMERADO != 0
    data['Total'] = True
    
    sums = data.groupby(['Total', 'AGLO_si', 'NOMAGLO'] + groupers + ['Q'] + vars_).size()
    sums = sums/frac
    sums.name = 'Personas'
    out = sums.reset_index()

    df_list += [out]
    
df = pd.concat(df_list)
df['Q'] = pd.to_datetime(df['Q'])
df10 = df.loc[df.Q.dt.year == 2010]

./../data/Pobreza/pobreza_0.01_q2010-02-15.csv
./../data/Pobreza/pobreza_0.01_q2010-05-15.csv
./../data/Pobreza/pobreza_0.01_q2010-08-15.csv
./../data/Pobreza/pobreza_0.01_q2010-11-15.csv
./../data/Pobreza/pobreza_0.01_q2011-02-15.csv
./../data/Pobreza/pobreza_0.01_q2011-05-15.csv
./../data/Pobreza/pobreza_0.01_q2011-08-15.csv
./../data/Pobreza/pobreza_0.01_q2011-11-15.csv


In [10]:
# Index(['index', 'RADIO_REF_ID', 'URP', 'DPTO', 'HOGAR_REF_ID', 'H15', 'H16',
#        'PERSONA_REF_ID', 'P02', 'P03', 'P09', 'P10', 'CONDACT', 'IX_TOT',
#        'CAT_OCUP', 'CAT_INAC', 'INGRESO', 'INGRESO_NLB', 'INGRESO_JUB',
#        'INGRESO_SBS', 'PP07K', 'P47T_persona', 'ANO4', 'Q', 'P0910',
#        'Grupo_Etario_3', 'Grupo_Etario_INDEC', 'Grupo_Etario_q10',
#        'P47T_hogar', 'CBA', 'CBT', 'CB_EQUIV', 'Pobreza', 'Indigencia',
#        'Pobreza_5usd', 'gap_pobreza', 'gap_indigencia', 'IDFRAC', 'PROV',
#        'NOMPROV', 'AGLOMERADO', 'Region'],
#       dtype='object')

## Personas. Total Pais. Poblacion por condicion de Pobreza / Indigencia

In [11]:
data = df10

for col in ['Pobreza', 'Indigencia']:
    display(Markdown('### '+col))
    tabla = data.groupby(['Q', col])['Personas'].sum().unstack().mean()
    tabla = data.groupby(['Q', col])['Personas'].sum().unstack().mean()
    tabla = pd.concat([tabla, round(100*tabla/tabla.sum(), 1)], axis = 1)
    tabla.columns = ['Cantidad', 'Porcentaje']

    tabla.loc['TOTAL'] = tabla.sum();
    
    display(tabla)

### Pobreza

,Cantidad,Porcentaje
Pobreza,,
False,23160800.0,57.9
True,16844600.0,42.1
TOTAL,40005400.0,100.0


### Indigencia

,Cantidad,Porcentaje
Indigencia,,
False,34532200.0,86.3
True,5473200.0,13.7
TOTAL,40005400.0,100.0


## Por AGLO SI/NO

In [66]:
# data = df10.loc[df10.AGLO_si == 1]

for col in ['Pobreza', 'Indigencia']:
    display(Markdown('### '+col))
#     tabla = data.groupby(['Q', col])['Personas'].sum().unstack().mean()    
    tabla = df10.groupby(['Q', col, 'AGLO_si'])['Personas'].sum().unstack([-2, -1]).mean()
    tabla = tabla.unstack()
    
    tabla_ = pd.concat([round(tabla/1e6, 2), round(100*tabla/tabla.sum(), 1)], axis = 1)
    tabla_.columns = pd.MultiIndex.from_product([['Cantidad (mill.)', 'Porcentaje (%)'], tabla.columns],
                               names = ['', 'AGLO_si'])
    
    tabla_.loc['TOTAL'] = tabla_.sum();
    
    display(tabla_)

### Pobreza

Cantidad (mill.)        Porcentaje (%)       
AGLOMERADO            False   True          False   True
Pobreza                                                 
False                  7.22  15.94           48.9   63.1
True                   7.54   9.31           51.1   36.9
TOTAL                 14.76  25.25          100.0  100.0

### Indigencia

Cantidad (mill.)        Porcentaje (%)       
AGLOMERADO            False   True          False   True
Indigencia                                              
False                 11.96  22.58           81.0   89.4
True                   2.80   2.67           19.0   10.6
TOTAL                 14.76  25.25          100.0  100.0

## Por AGLOMERADOS

In [71]:
# data = df10.loc[df10.AGLO_si == 1]

for col in ['Pobreza', 'Indigencia']:
    display(Markdown('### '+col))
#     tabla = data.groupby(['Q', col])['Personas'].sum().unstack().mean()    
    tabla = df10.groupby(['Q', col, 'NOMAGLO'])['Personas'].sum().unstack([-2, -1]).mean()
    tabla = tabla.unstack()
    
    tabla_ = pd.concat([round(tabla/1e3, 1), round(100*tabla/tabla.sum(), 1)], axis = 1)
    tabla_.columns = pd.MultiIndex.from_product([['Cantidad (miles)', 'Porcentaje (%)'], tabla.columns],
                               names = ['', 'AGLOMERADO'])
    
    tabla_.loc['TOTAL'] = tabla_.sum();
    
    display(tabla_.stack(0).T)

### Pobreza

Pobreza                       False                            True  \
                   Cantidad (miles) Porcentaje (%) Cantidad (miles)   
AGLOMERADO                                                            
Bahia Blanca                  201.0           68.4             92.7   
CABA                         2398.4           83.6            469.4   
Catamarca                     105.1           59.9             70.5   
Cdro Rivadavia                126.6           79.1             33.4   
Concordia                      75.4           50.1             75.1   
Cordoba                       949.1           65.6            498.4   
Corrientes                    173.2           50.9            166.8   
Formosa                       111.4           50.5            109.0   
Fuera de aglom.              7223.0           48.9           7537.6   
GBA                          6425.7           60.2           4245.5   
Jujuy                         172.7           55.6            137.7   
La Plata                      537.2           68.0            252.2   
La Rioja                       89.1           51.0             85.8   
Mar del Plata                 403.4           66.6            202.2   
Mendoza                       560.8           60.3            368.8   
Neuquen                       231.0           66.7            115.4   
Parana                        172.1           63.0            101.1   
Posadas                       150.6           51.0            144.4   
Rawson-Trelew                  83.8           71.2             33.9   
Resistencia                   188.4           49.3            194.2   
Rio Cuarto                    100.2           66.9             49.6   
Rio Gallegos                   81.0           75.7             26.0   
Rosario                       761.1           64.2            425.2   
Salta                         304.6           55.1            248.4   
San Juan                      207.0           51.4            195.9   
San Luis                       95.9           52.1             88.0   
San Nicolas                   108.8           60.8             70.0   
Santa Fe                      278.0           58.2            199.6   
Santa Rosa                     75.2           67.0             37.1   
Sgo del Estero                182.6           50.7            177.9   
Tucuman                       452.1           55.7            359.2   
Ushuaia-Rio Grande             88.9           86.8             13.5   
Viedma-Patagones               47.5           70.5             19.9   

Pobreza                                      TOTAL                 
                   Porcentaje (%) Cantidad (miles) Porcentaje (%)  
AGLOMERADO                                                         
Bahia Blanca                 31.6            293.7          100.0  
CABA                         16.4           2867.8          100.0  
Catamarca                    40.1            175.6          100.0  
Cdro Rivadavia               20.9            160.0          100.0  
Concordia                    49.9            150.5          100.0  
Cordoba                      34.4           1447.5          100.0  
Corrientes                   49.1            340.0          100.0  
Formosa                      49.5            220.4          100.0  
Fuera de aglom.              51.1          14760.6          100.0  
GBA                          39.8          10671.2          100.0  
Jujuy                        44.4            310.4          100.0  
La Plata                     32.0            789.4          100.0  
La Rioja                     49.0            174.9          100.0  
Mar del Plata                33.4            605.6          100.0  
Mendoza                      39.7            929.6          100.0  
Neuquen                      33.3            346.4          100.0  
Parana                       37.0            273.2          100.0  
Posadas                      49.0            295.0          100.0  
Rawson-Trelew  

### Indigencia

Indigencia                    False                            True  \
                   Cantidad (miles) Porcentaje (%) Cantidad (miles)   
AGLOMERADO                                                            
Bahia Blanca                  268.0           91.3             25.7   
CABA                         2755.0           96.1            112.9   
Catamarca                     156.0           88.8             19.6   
Cdro Rivadavia                152.2           95.1              7.8   
Concordia                     125.9           83.7             24.6   
Cordoba                      1310.4           90.5            137.1   
Corrientes                    273.9           80.6             66.1   
Formosa                       179.2           81.3             41.2   
Fuera de aglom.             11957.0           81.0           2803.6   
GBA                          9514.6           89.2           1156.6   
Jujuy                         266.3           85.8             44.1   
La Plata                      717.8           90.9             71.6   
La Rioja                      155.4           88.9             19.5   
Mar del Plata                 558.0           92.1             47.6   
Mendoza                       831.5           89.4             98.1   
Neuquen                       317.5           91.7             28.8   
Parana                        250.1           91.5             23.1   
Posadas                       237.9           80.7             57.1   
Rawson-Trelew                 112.4           95.5              5.4   
Resistencia                   310.0           81.0             72.6   
Rio Cuarto                    136.5           91.1             13.4   
Rio Gallegos                  100.0           93.5              7.0   
Rosario                      1064.3           89.7            122.0   
Salta                         478.0           86.4             75.0   
San Juan                      342.3           85.0             60.6   
San Luis                      157.5           85.6             26.4   
San Nicolas                   156.2           87.3             22.7   
Santa Fe                      408.1           85.5             69.5   
Santa Rosa                    101.2           90.1             11.2   
Sgo del Estero                285.9           79.3             74.6   
Tucuman                       690.0           85.1            121.3   
Ushuaia-Rio Grande            101.0           98.7              1.4   
Viedma-Patagones               62.3           92.4              5.1   

Indigencia                                   TOTAL                 
                   Porcentaje (%) Cantidad (miles) Porcentaje (%)  
AGLOMERADO                                                         
Bahia Blanca                  8.7            293.7          100.0  
CABA                          3.9           2867.9          100.0  
Catamarca                    11.2            175.6          100.0  
Cdro Rivadavia                4.9            160.0          100.0  
Concordia                    16.3            150.5          100.0  
Cordoba                       9.5           1447.5          100.0  
Corrientes                   19.4            340.0          100.0  
Formosa                      18.7            220.4          100.0  
Fuera de aglom.              19.0          14760.6          100.0  
GBA                          10.8          10671.2          100.0  
Jujuy                        14.2            310.4          100.0  
La Plata                      9.1            789.4          100.0  
La Rioja                     11.1            174.9          100.0  
Mar del Plata                 7.9            605.6          100.0  
Mendoza                      10.6            929.6          100.0  
Neuquen                       8.3            346.3          100.0  
Parana                        8.5            273.2          100.0  
Posadas                      19.3            295.0          100.0  
Rawson-Trelew  

## Por PROVINCIAS

In [72]:
# data = df10.loc[df10.AGLO_si == 1]

for col in ['Pobreza', 'Indigencia']:
    display(Markdown('### '+col))
#     tabla = data.groupby(['Q', col])['Personas'].sum().unstack().mean()    
    tabla = df10.groupby(['Q', col, 'NOMPROV'])['Personas'].sum().unstack([-2, -1]).mean()
    tabla = tabla.unstack()
    
    tabla_ = pd.concat([round(tabla/1e3, 1), round(100*tabla/tabla.sum(), 1)], axis = 1)
    tabla_.columns = pd.MultiIndex.from_product([['Cantidad (miles)', 'Porcentaje (%)'], tabla.columns],
                               names = ['', 'PROVINCIA'])
    
    tabla_.loc['TOTAL'] = tabla_.sum();
    
    display(tabla_.stack(0).T)

### Pobreza

Pobreza                                    False                 \
                                Cantidad (miles) Porcentaje (%)   
PROVINCIA                                                         
Buenos Aires                              9559.9           61.2   
Catamarca                                  181.4           48.7   
Chaco                                      386.1           36.8   
Chubut                                     365.6           72.4   
Ciudad Autónoma de Buenos Aires           2399.2           83.6   
Corrientes                                 404.4           41.2   
Córdoba                                   2023.0           61.6   
Entre Ríos                                 652.0           52.8   
Formosa                                    199.4           37.3   
Jujuy                                      310.5           46.4   
La Pampa                                   198.1           62.2   
La Rioja                                   147.9           43.8   
Mendoza                                    914.4           52.4   
Misiones                                   414.8           37.5   
Neuquén                                    344.6           62.7   
Río Negro                                  417.2           66.2   
Salta                                      525.8           43.3   
San Juan                                   292.5           42.9   
San Luis                                   226.7           52.4   
Santa Cruz                                 198.6           71.5   
Santa Fe                                  1888.8           59.5   
Santiago del Estero                        325.7           37.8   
Tierra del Fuego                            99.7           83.4   
Tucumán                                    684.5           47.6   

Pobreza                                     True                 \
                                Cantidad (miles) Porcentaje (%)   
PROVINCIA                                                         
Buenos Aires                              6056.4           38.8   
Catamarca                                  191.5           51.3   
Chaco                                      662.8           63.2   
Chubut                                     139.2           27.6   
Ciudad Autónoma de Buenos Aires            469.6           16.4   
Corrientes                                 577.4           58.8   
Córdoba                                   1262.5           38.4   
Entre Ríos                                 583.7           47.2   
Formosa                                    335.4           62.7   
Jujuy                                      358.8           53.6   
La Pampa                                   120.3           37.8   
La Rioja                                   190.1           56.2   
Mendoza                                    830.0           47.6   
Misiones                                   691.6           62.5   
Neuquén                                    204.8           37.3   
Río Negro                                  213.4           33.8   
Salta                                      689.8           56.7   
San Juan                                   388.6           57.1   
San Luis                                   205.5           47.6   
Santa Cruz                                  79.0           28.5   
Santa Fe                                  1284.4           40.5   
Santiago del Estero                        536.4           62.2   
Tierra del Fuego                            19.8           16.6   
Tucumán                                    753.5           52.4   

Pobreza                                    TOTAL                 
                                Cantidad (miles) Porcentaje (%)  
PROVINCIA                                                        
Buenos Aires                             15616.3          100.0  
Catamarca                                  372.9          100.0  
Chaco                                     1048.9  

### Indigencia

Indigencia                                 False                 \
                                Cantidad (miles) Porcentaje (%)   
PROVINCIA                                                         
Buenos Aires                             13965.6           89.4   
Catamarca                                  299.3           80.3   
Chaco                                      740.6           70.6   
Chubut                                     474.5           94.0   
Ciudad Autónoma de Buenos Aires           2756.0           96.1   
Corrientes                                 723.1           73.6   
Córdoba                                   2932.9           89.3   
Entre Ríos                                1059.7           85.8   
Formosa                                    364.2           68.1   
Jujuy                                      534.0           79.8   
La Pampa                                   282.1           88.6   
La Rioja                                   282.3           83.5   
Mendoza                                   1510.5           86.6   
Misiones                                   776.4           70.2   
Neuquén                                    492.1           89.6   
Río Negro                                  573.8           91.0   
Salta                                      936.9           77.1   
San Juan                                   553.8           81.3   
San Luis                                   373.4           86.4   
Santa Cruz                                 258.8           93.2   
Santa Fe                                  2766.9           87.2   
Santiago del Estero                        607.1           70.4   
Tierra del Fuego                           117.1           98.0   
Tucumán                                   1151.2           80.1   

Indigencia                                  True                 \
                                Cantidad (miles) Porcentaje (%)   
PROVINCIA                                                         
Buenos Aires                              1650.6           10.6   
Catamarca                                   73.6           19.7   
Chaco                                      308.4           29.4   
Chubut                                      30.2            6.0   
Ciudad Autónoma de Buenos Aires            112.9            3.9   
Corrientes                                 258.7           26.4   
Córdoba                                    352.6           10.7   
Entre Ríos                                 176.0           14.2   
Formosa                                    170.8           31.9   
Jujuy                                      135.3           20.2   
La Pampa                                    36.3           11.4   
La Rioja                                    55.7           16.5   
Mendoza                                    233.9           13.4   
Misiones                                   330.0           29.8   
Neuquén                                     57.3           10.4   
Río Negro                                   56.8            9.0   
Salta                                      278.7           22.9   
San Juan                                   127.3           18.7   
San Luis                                    58.8           13.6   
Santa Cruz                                  18.8            6.8   
Santa Fe                                   406.3           12.8   
Santiago del Estero                        255.0           29.6   
Tierra del Fuego                             2.4            2.0   
Tucumán                                    286.8           19.9   

Indigencia                                 TOTAL                 
                                Cantidad (miles) Porcentaje (%)  
PROVINCIA                                                        
Buenos Aires                             15616.2          100.0  
Catamarca                                  372.9          100.0  
Chaco                                     1049.0  